<a href="https://colab.research.google.com/github/JAYANTHNITW/NLP/blob/main/MultichannelCNNArchitecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense,Embedding,Dropout,LSTM,GRU,Flatten,Conv1D,Input,GlobalMaxPool1D,Concatenate

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train = pd.read_csv("Dataset-Multitask1.csv")
df_train.tail()

,Unnamed: 0,Text,Sarcasm,Stress
7284,1150,"@AngelLamuno @lunarossa Relax mate, we have a ...",0,0
7285,1151,"This doesn't mean that you relax on COVID 19, ...",0,0
7286,1152,Melbourne to relax third lockdown as no new CO...,0,0
7287,1153,JUST IN: Mayors vote 9-8 to put Metro Manila u...,0,0
7288,1154,"You can return to your social apps and news, b...",1,0


In [ ]:
df1 = pd.read_csv("Dataset-Multitask1.csv")
df1.shape

(7289, 4)

In [ ]:
df2 = pd.read_csv("Dataset-Multitask2.csv")
df2.shape

FileNotFoundError: ignored

In [ ]:
df3 = pd.read_csv("Dataset-Multitask3.csv")
df3.shape

In [ ]:
df4 = pd.read_csv("Dataset-Multitask4.csv")
df4.shape

In [ ]:
# df_test = pd.read_csv("Dataset-Multitask1.csv")
# df_test.tail()

In [ ]:
df_train = df_train.drop(['Unnamed: 0'],axis=1)
# df_test = df_test.drop(['Unnamed: 0'],axis=1)

In [ ]:
#df_train = pd.concat([df_train,df_test],axis=0)

In [ ]:
df_train.shape

(7289, 3)

# TEXT Preprocessing

## 1. Lower Casing

In [ ]:
df_train['Text'] = df_train['Text'].str.lower()

## 2. HTML Tags

In [ ]:
import re
def remove_html_tags(text):
  pattern = re.compile('<.*?>')
  return pattern.sub(r'',text)

In [ ]:
df_train['Text'] = df_train['Text'].apply(remove_html_tags)

## Removing URL's in Text

In [ ]:
def remove_url(text):
  pattern = re.compile(r'https?://\S+|www\.\/S+')
  return pattern.sub(r'',text)

In [ ]:
df_train['Text'] = df_train['Text'].apply(remove_url)

## Removing Punctuations

In [ ]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
exclude = string.punctuation

In [ ]:
def remove_punc(text):
  return text.translate(str.maketrans('','',exclude))

In [ ]:
df_train['Text'] = df_train['Text'].apply(remove_punc)

## 5. Spelling Corrections

In [ ]:
from textblob import TextBlob

In [ ]:
def correct_spelling(text):
  blob = TextBlob(text)
  corrected_text = blob.correct()
  return str(corrected_text)

## 6. Stemming

In [ ]:
from nltk import PorterStemmer

In [ ]:
stemmer = PorterStemmer()

In [ ]:
df_train['Text'] = df_train['Text'].apply(lambda x:stemmer.stem(x))

## 7. Removing Stopwords

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
sw_list = stopwords.words('english')

df_train['Text'].apply(lambda x:[item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

0       peerreviewed article systematically deconstruc...
1       dealing stress children youth living diabetes ...
2       pandemic living stressful everyone even who’ve...
3       stress leave tears insomnia greater victoria t...
4       stress leave tears insomnia greater victoria t...
                              ...                        
7284    angellamuno lunarossa relax mate real covid19 ...
7285        doesnt mean relax covid 19 still us wearamask
7286    melbourne relax third lockdown new covid19 cas...
7287    mayors vote 98 put metro manila mgcq gcq march...
7288    return social apps news first make time availa...
Name: Text, Length: 7289, dtype: object

## 8. Tokenization

In [ ]:
import nltk
from nltk import word_tokenize

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df_train['Text'].apply(lambda x:word_tokenize(x))

0       [peerreviewed, article, that, systematically, ...
1       [dealing, with, stress, for, children, and, yo...
2       [pandemic, living, is, stressful, for, everyon...
3       [stress, leave, tears, and, insomnia, greater,...
4       [stress, leave, tears, and, insomnia, greater,...
                              ...                        
7284    [angellamuno, lunarossa, relax, mate, we, have...
7285    [this, doesnt, mean, that, you, relax, on, cov...
7286    [melbourne, to, relax, third, lockdown, as, no...
7287    [just, in, mayors, vote, 98, to, put, metro, m...
7288    [you, can, return, to, your, social, apps, and...
Name: Text, Length: 7289, dtype: object

In [ ]:
df_train['Text'] = df_train['Text'].apply(lambda x:word_tokenize(x))

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train['Text'])

In [ ]:
x = tokenizer.texts_to_sequences(df_train['Text'])

In [ ]:
df_train['Text'].shape

(7289,)

In [ ]:
# Vocabulary based on tokenization
vocabulary = tokenizer.word_index
print(len(vocabulary))

5082


In [ ]:
import gensim

In [ ]:
!wget nlp.stanford.edu/data/glove.6B.zip

--2023-10-09 10:34:07--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-10-09 10:34:07--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-10-09 10:34:08--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec

In [ ]:
glove_input_file = "glove.6B.100d.txt"

In [ ]:
glove_output_file = "word2vec.txt"

In [ ]:
glove2word2vec(glove_input_file,glove_output_file)

<ipython-input-38-13f14740c424>:1: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file,glove_output_file)


(400000, 100)

In [ ]:
maxlen = 25
from keras.utils import pad_sequences
lines_paded = pad_sequences(x,maxlen=25,padding='post')

In [ ]:
y1 = np.array(df_train['Sarcasm'])
y2 = np.array(df_train['Stress'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train_task1, y_val_task1, y_train_task2, y_val_task2 = train_test_split(
    lines_paded, y1, y2, test_size=0.2, random_state=42)

In [ ]:
print(f"x_train shape {X_train.shape}")
print(f"x_val shape {X_val.shape}")
print(f"y_train_task1 shape {y_train_task1.shape}")
print(f"y_train_task2 shape {y_train_task2.shape}")
print(f"y_val_task1 shape {y_val_task1.shape}")
print(f"y_val_task2 shape {y_val_task2.shape}")

x_train shape (5831, 25)
x_val shape (1458, 25)
y_train_task1 shape (5831,)
y_train_task2 shape (5831,)
y_val_task1 shape (1458,)
y_val_task2 shape (1458,)


In [ ]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from gensim.models import KeyedVectors

glove_model = KeyedVectors.load_word2vec_format(glove_output_file, binary=False)

vocab_size = len(vocabulary) + 1  # Add 1 for unknown words

embedding_dim = 100  # Adjust the embedding dimension based on your GloVe model
max_length = 25
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in vocabulary.items():
    if word in glove_model:
        embedding_matrix[i] = glove_model[word]

## Multitask CNN architecutre

In [ ]:
from sklearn.metrics import f1_score,precision_score

In [ ]:
from keras.optimizers import Adam
sequence_len= 25
embedding_dim= 100
input_shape = (sequence_len,)
input_layer = Input(shape=input_shape)
embedding_layer = Embedding(input_dim=vocab_size,
                            output_dim=embedding_dim,
                            weights=[embedding_matrix],
                            trainable=False)(input_layer)
# channel 1
dropout_1 = Dropout(0.5)(embedding_layer)
conv_1d_1 = Conv1D(64,5,activation='relu')(dropout_1)
dropout_2 = Dropout(0.3)(conv_1d_1)
gmp1d_1 = GlobalMaxPool1D()(dropout_2)
dense_control_1 = Dense(64,activation='relu')(gmp1d_1)

#  Channel
dropout_3 = Dropout(0.5)(embedding_layer)
conv_1d_2 = Conv1D(64,5,activation='relu')(dropout_3)
dropout_4 = Dropout(0.3)(conv_1d_2)
gmp1d_2 = GlobalMaxPool1D()(dropout_4)

# # For Task3
# dropout_5 = Dropout(0.5)(embedding_layer)
# conv_1d_3 = Conv1D(64,activation='relu')(dropout_5)
# dropout_6 = Dropout(0.3)(conv_1d_3)
# gmp1d_3 = GlobalMaxPool1D()(dropout_6)

merged = Concatenate(axis=-1)([gmp1d_1,gmp1d_2])

dense1 = Dense(64,activation='relu')(merged)
output_sarcasm = Dense(1,activation='sigmoid',name='output_sarcasm')(dense1)
output_stress = Dense(1,activation='sigmoid',name='output_stress')(dense1)

model = Model(inputs=input_layer,outputs=[output_sarcasm,output_stress])
high_learning_rate = 0.1
model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=high_learning_rate),metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 25)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 25, 100)              508300    ['input_1[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 25, 100)              0         ['embedding[0][0]']           
                                                                                                  
 dropout_2 (Dropout)         (None, 25, 100)              0         ['embedding[0][0]']           
                                                                                              

In [ ]:
from keras.utils import plot_model
plot_model(model)

In [ ]:
# Load TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# Create a function to build a TensorBoard callback
# def create_tensorborad_callback():
#   # Create a log directory for storing TensorBoard logs
#   logdir = os.path.join('/content/drive/MyDrive/logs',
#                         # Make it so the logs get tracked whenever we run an experiment
#                         datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
#   return tf.keras.callbacks.TensorBoard(logdir)

In [ ]:
import os,datetime
import tensorflow as tf

In [ ]:
# tensorboard = create_tensorborad_callback(),callbacks=[tensorboard]

In [ ]:
history = model.fit(X_train,[y_train_task1,y_train_task2],validation_data=(X_val,[y_val_task1,y_val_task2]),epochs=15,batch_size=32)

Epoch 1/15
183/183 [==============================] - 14s 13ms/step - loss: 2.1574 - output_sarcasm_loss: 0.6915 - output_stress_loss: 1.4659 - output_sarcasm_accuracy: 0.9365 - output_stress_accuracy: 0.8242 - val_loss: 0.5448 - val_output_sarcasm_loss: 0.2055 - val_output_stress_loss: 0.3394 - val_output_sarcasm_accuracy: 0.9472 - val_output_stress_accuracy: 0.8649
Epoch 2/15
183/183 [==============================] - 1s 8ms/step - loss: 0.6048 - output_sarcasm_loss: 0.2256 - output_stress_loss: 0.3792 - output_sarcasm_accuracy: 0.9441 - output_stress_accuracy: 0.8553 - val_loss: 0.5564 - val_output_sarcasm_loss: 0.2099 - val_output_stress_loss: 0.3464 - val_output_sarcasm_accuracy: 0.9472 - val_output_stress_accuracy: 0.8621
Epoch 3/15
183/183 [==============================] - 1s 7ms/step - loss: 0.6318 - output_sarcasm_loss: 0.2227 - output_stress_loss: 0.4091 - output_sarcasm_accuracy: 0.9443 - output_stress_accuracy: 0.8573 - val_loss: 0.5935 - val_output_sarcasm_loss: 0.2081 - 

In [ ]:
#model.fit(X_train,[y_train_task1,y_train_task2],validation_data=(X_val,[y_val_task1,y_val_task2]),epochs=15,batch_size=32)

In [ ]:
y_pred_task1,y_pred_task2 = model.predict(X_val)

46/46 [==============================] - 0s 2ms/step


In [ ]:
threshold=0.5
y_pred_binary1 = np.where(y_pred_task1>=threshold ,1,0)
y_pred_binary2 = np.where(y_pred_task2>=threshold ,1,0)
# print(y_pred_binary1)

In [ ]:
f1_score_Sarcasm = f1_score(y_val_task1,y_pred_binary1)
print(f1_score_Sarcasm)

0.0


In [ ]:
ps_sarcasm = precision_score(y_val_task1,y_pred_binary1)
print(ps_sarcasm)

0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
f1_score_Stress = f1_score(y_val_task2,y_pred_binary2)
print(f1_score_Stress )

0.9159851301115242


In [ ]:
ps_stress = precision_score(y_val_task2,y_pred_binary2)
print(ps_stress)

0.8449931412894376


## GRU CNN GRU WITH ATTENTION LAYERS

In [ ]:
from keras.src.layers import Concatenate
from keras.layers import Bidirectional,GlobalMaxPool1D,Attention,RepeatVector
from keras.optimizers import Adam
sequence_len=25
embedding_dim=100
input_shape=(sequence_len,)
input_layer = Input(shape=input_shape)
embedding_layer = Embedding(vocab_size,embedding_dim,weights=[embedding_matrix],trainable=False)(input_layer)
# Conv layer + Maxpool layer *3
conv_layer=Conv1D(64,5,activation='relu')(embedding_layer)
conv_layer = GlobalMaxPool1D()(conv_layer)

# Lstm for task1
lstm_layer1 = Bidirectional(GRU(64,return_sequences=True))(embedding_layer)
attention_layer1 = Attention()([lstm_layer1, lstm_layer1])
attended_lstm1 = Concatenate(axis=-1)([lstm_layer1, attention_layer1])
lstm_layer1 = GlobalMaxPool1D()(attended_lstm1)

# Lstm for task2
lstm_layer2 = Bidirectional(GRU(64,return_sequences=True))(embedding_layer)
attention_layer2 = Attention()([lstm_layer2, lstm_layer2])
attended_lstm2 = Concatenate(axis=-1)([lstm_layer2, attention_layer2])
lstm_layer2 = GlobalMaxPool1D()(attended_lstm2)
#Reshaping
conv_layer_reshaped = RepeatVector(sequence_len)(conv_layer)
lstm_layer_reshaped1 = RepeatVector(sequence_len)(lstm_layer1)
lstm_layer_reshaped2 = RepeatVector(sequence_len)(lstm_layer2)
# concatenation
concatenated1 = Concatenate(axis=-1)([conv_layer_reshaped,lstm_layer_reshaped1])
concatenated2 = Concatenate(axis=-1)([conv_layer_reshaped,lstm_layer_reshaped2])
# Flatten the concatenated output to match the shape for binary classification
flattened_output1 = Flatten()(concatenated1)
flattened_output2 = Flatten()(concatenated2)

output1 = Dense(1,activation='sigmoid',name='sarcasm')(flattened_output1)
output2 = Dense(1,activation='sigmoid',name='stress')(flattened_output2)

model = Model(inputs=input_layer, outputs=[output1,output2])
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.1), metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 25)]                 0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 25, 100)              508300    ['input_2[0][0]']             
                                                                                                  
 bidirectional (Bidirection  (None, 25, 128)              63744     ['embedding_1[0][0]']         
 al)                                                                                              
                                                                                                  
 bidirectional_1 (Bidirecti  (None, 25, 128)              63744     ['embedding_1[0][0]']   

In [ ]:
model.fit(X_train,[y_train_task1,y_train_task2],validation_data=(X_val,[y_val_task1,y_val_task2]),epochs=15,batch_size=32)

Epoch 1/15
183/183 [==============================] - 17s 37ms/step - loss: 26.3246 - sarcasm_loss: 14.8189 - stress_loss: 11.5057 - sarcasm_accuracy: 0.8928 - stress_accuracy: 0.8429 - val_loss: 25.8982 - val_sarcasm_loss: 17.8886 - val_stress_loss: 8.0095 - val_sarcasm_accuracy: 0.9396 - val_stress_accuracy: 0.7826
Epoch 2/15
183/183 [==============================] - 7s 40ms/step - loss: 22.4928 - sarcasm_loss: 15.6928 - stress_loss: 6.8000 - sarcasm_accuracy: 0.9010 - stress_accuracy: 0.8966 - val_loss: 24.4217 - val_sarcasm_loss: 19.7823 - val_stress_loss: 4.6394 - val_sarcasm_accuracy: 0.9444 - val_stress_accuracy: 0.9390
Epoch 3/15
183/183 [==============================] - 5s 28ms/step - loss: 20.3313 - sarcasm_loss: 15.1281 - stress_loss: 5.2032 - sarcasm_accuracy: 0.9034 - stress_accuracy: 0.9189 - val_loss: 16.8994 - val_sarcasm_loss: 9.8216 - val_stress_loss: 7.0777 - val_sarcasm_accuracy: 0.8992 - val_stress_accuracy: 0.8882
Epoch 4/15
183/183 [============================

In [ ]:
y_pred_task1,y_pred_task2 = model.predict(X_val)

46/46 [==============================] - 1s 4ms/step


In [ ]:
threshold=0.5
y_pred_binary1 = np.where(y_pred_task1>=threshold ,1,0)
y_pred_binary2 = np.where(y_pred_task2>=threshold ,1,0)
print(y_pred_binary1)

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


In [ ]:
f1_score_Sarcasm = f1_score(y_val_task1,y_pred_binary1)
print(f1_score_Sarcasm)

0.07058823529411765


In [ ]:
ps_sarcasm = precision_score(y_val_task1,y_pred_binary1)
print(ps_sarcasm)

0.375


In [ ]:
f1_score_Stress = f1_score(y_val_task2,y_pred_binary2)
print(f1_score_Stress )

0.9600000000000001


In [ ]:
ps_stress = precision_score(y_val_task2,y_pred_binary2)
print(ps_stress)

0.9863013698630136


In [ ]:
from keras.src.layers import Concatenate
from keras.optimizers import Adam
sequence_len=25
embedding_dim=100
input_shape=(sequence_len,)
input_layer = Input(shape=input_shape)
embedding_layer = Embedding(vocab_size,embedding_dim,weights=[embedding_matrix],trainable=False)(input_layer)

BiGRU_layer = Bidirectional(GRU(64,return_sequences=True))(embedding_layer)
attention_layer = Attention()([BiGRU_layer,BiGRU_layer])
concatenated_layer = Concatenate(axis=-1)([attention_layer,BiGRU_layer])
gmp_layer = GlobalMaxPool1D()(concatenated_layer)

dense_layer1 = Dense(64,activation='relu')(gmp_layer)
dense_layer2 = Dense(64,activation='relu')(gmp_layer)

output_sarcasm = Dense(1,activation='sigmoid',name='sarcasm')(dense_layer1)
concatenated_for_stress = Concatenate(axis=-1)([dense_layer2,output_sarcasm])
output_stress = Dense(1,activation='sigmoid',name='stress')(concatenated_for_stress)

model = Model(inputs=input_layer,outputs=[output_sarcasm,output_stress])
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.1), metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 25)]                 0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 25, 100)              508300    ['input_3[0][0]']             
                                                                                                  
 bidirectional_2 (Bidirecti  (None, 25, 128)              63744     ['embedding_2[0][0]']         
 onal)                                                                                            
                                                                                                  
 attention_2 (Attention)     (None, 25, 128)              0         ['bidirectional_2[0][0]'

In [ ]:
model.fit(X_train,[y_train_task1,y_train_task2],validation_data=(X_val,[y_val_task1,y_val_task2]),epochs=15,batch_size=32)

Epoch 1/15
183/183 [==============================] - 8s 17ms/step - loss: 0.8799 - sarcasm_loss: 0.3234 - stress_loss: 0.5565 - sarcasm_accuracy: 0.9386 - stress_accuracy: 0.8343 - val_loss: 0.6349 - val_sarcasm_loss: 0.2064 - val_stress_loss: 0.4285 - val_sarcasm_accuracy: 0.9472 - val_stress_accuracy: 0.8450
Epoch 2/15
183/183 [==============================] - 2s 13ms/step - loss: 0.6509 - sarcasm_loss: 0.2121 - stress_loss: 0.4388 - sarcasm_accuracy: 0.9451 - stress_accuracy: 0.8407 - val_loss: 0.6391 - val_sarcasm_loss: 0.2066 - val_stress_loss: 0.4325 - val_sarcasm_accuracy: 0.9472 - val_stress_accuracy: 0.8450
Epoch 3/15
183/183 [==============================] - 2s 11ms/step - loss: 0.6527 - sarcasm_loss: 0.2144 - stress_loss: 0.4383 - sarcasm_accuracy: 0.9451 - stress_accuracy: 0.8407 - val_loss: 0.7013 - val_sarcasm_loss: 0.2124 - val_stress_loss: 0.4889 - val_sarcasm_accuracy: 0.9472 - val_stress_accuracy: 0.8450
Epoch 4/15
183/183 [==============================] - 2s 13ms

In [ ]:
y_pred_task1,y_pred_task2 = model.predict(X_val)

46/46 [==============================] - 1s 5ms/step


In [ ]:
print(y_pred_task1.max())

0.15533917


In [ ]:
threshold=0.5
y_pred_binary1 = np.where(y_pred_task1>=threshold ,1,0)
y_pred_binary2 = np.where(y_pred_task2>=threshold ,1,0)
print(y_pred_binary1)

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


In [ ]:
y_val_task1[:100]

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [ ]:
y_pred_binary1[:10]

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

In [ ]:
f1_score_Sarcasm = f1_score(y_val_task1,y_pred_binary1)
print(f1_score_Sarcasm)

0.0


In [ ]:
ps_sarcasm = precision_score(y_val_task1,y_pred_binary1)
print(ps_sarcasm)

0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
f1_score_Stress = f1_score(y_val_task2,y_pred_binary2)
print(f1_score_Stress )

0.9159851301115242


In [ ]:
ps_stress = precision_score(y_val_task2,y_pred_binary2)
print(ps_stress)

0.8449931412894376


In [ ]:
# !tensorboard dev upload \
# --logdir "/content/drive/MyDrive/logs" \
# --name "Sarcasm and Stress Multichannel CNN with 4 datasets" \
# --one_shot

In [ ]:
from keras.optimizers import Adam
from keras.layers import Bidirectional
inp = Input(shape=(25,))
x = Embedding(vocab_size, 100, weights=[embedding_matrix],trainable=False)(inp)
x = LSTM(1, return_sequences=True)(x)#return_sequences=True
x = GlobalMaxPool1D()(x)
# x = Dense(50, activation="relu")(x)
# x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.1), metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train_task1,validation_data=(X_val,y_val_task1),epochs=15,batch_size=32)

Epoch 1/15
183/183 [==============================] - 7s 13ms/step - loss: 0.2202 - accuracy: 0.9451 - val_loss: 0.2067 - val_accuracy: 0.9472
Epoch 2/15
183/183 [==============================] - 2s 11ms/step - loss: 0.2148 - accuracy: 0.9451 - val_loss: 0.2072 - val_accuracy: 0.9472
Epoch 3/15
183/183 [==============================] - 2s 10ms/step - loss: 0.2143 - accuracy: 0.9451 - val_loss: 0.2068 - val_accuracy: 0.9472
Epoch 4/15
183/183 [==============================] - 2s 9ms/step - loss: 0.2152 - accuracy: 0.9451 - val_loss: 0.2073 - val_accuracy: 0.9472
Epoch 5/15
183/183 [==============================] - 2s 11ms/step - loss: 0.2157 - accuracy: 0.9451 - val_loss: 0.2086 - val_accuracy: 0.9472
Epoch 6/15
183/183 [==============================] - 2s 10ms/step - loss: 0.2146 - accuracy: 0.9451 - val_loss: 0.2069 - val_accuracy: 0.9472
Epoch 7/15
183/183 [==============================] - 1s 7ms/step - loss: 0.2148 - accuracy: 0.9451 - val_loss: 0.2071 - val_accuracy: 0.9472
E

In [ ]:
y_pred_task1 = model.predict(X_val)

46/46 [==============================] - 0s 2ms/step


In [ ]:
import numpy as np

threshold = 0.5  # Adjust this threshold as needed
y_pred_binary = np.where(y_pred_task1 >= threshold, 1, 0)
print(y_pred_binary[:10])

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


In [ ]:
f1_score_Sarcasm = f1_score(y_val_task1,y_pred_binary)
print(f1_score_Sarcasm)

0.10084033613445378


In [ ]:
ps_sarcasm = precision_score(y_val_task1,y_pred_binary1)
print(ps_sarcasm)

0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model.fit(X_train,y_train_task2,validation_data=(X_val,y_val_task2),epochs=15,batch_size=32)

Epoch 1/15
183/183 [==============================] - 1s 7ms/step - loss: 0.5161 - accuracy: 0.8180 - val_loss: 0.4368 - val_accuracy: 0.8450
Epoch 2/15
183/183 [==============================] - 2s 9ms/step - loss: 0.4399 - accuracy: 0.8407 - val_loss: 0.4465 - val_accuracy: 0.8450
Epoch 3/15
183/183 [==============================] - 1s 8ms/step - loss: 0.4421 - accuracy: 0.8407 - val_loss: 0.4314 - val_accuracy: 0.8450
Epoch 4/15
183/183 [==============================] - 1s 6ms/step - loss: 0.4417 - accuracy: 0.8407 - val_loss: 0.4314 - val_accuracy: 0.8450
Epoch 5/15
183/183 [==============================] - 1s 6ms/step - loss: 0.4409 - accuracy: 0.8407 - val_loss: 0.4319 - val_accuracy: 0.8450
Epoch 6/15
183/183 [==============================] - 1s 8ms/step - loss: 0.4402 - accuracy: 0.8407 - val_loss: 0.4348 - val_accuracy: 0.8450
Epoch 7/15
183/183 [==============================] - 2s 9ms/step - loss: 0.4407 - accuracy: 0.8407 - val_loss: 0.4315 - val_accuracy: 0.8450
Epoch 

In [ ]:
y_pred_task2 = model.predict(X_val)

46/46 [==============================] - 0s 2ms/step


In [ ]:
y_pred_binary2 = np.where(y_pred_task2 >= threshold, 1, 0)
print(y_pred_binary2[:10])

[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]


In [ ]:
f1_score_Stress = f1_score(y_val_task2,y_pred_binary2)
print(f1_score_Stress)

0.9159851301115242


In [ ]:
ps_stress = precision_score(y_val_task2,y_pred_binary2)
print(ps_stress)

0.8449931412894376
